# **Build BioMistral Medical RAG Chatbot using BioMistral Open Source LLM

Load the Google Drive


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install langchain

  Using cached chromadb-0.6.3-py3-none-any.whl.metadata (6.8 kB)
  Using cached llama_cpp_python-0.3.7-cp311-cp311-linux_x86_64.whl
  Using cached langchain_community-0.3.16-py3-none-any.whl.metadata (2.9 kB)
  Using cached build-1.2.2.post1-py3-none-any.whl.metadata (6.5 kB)
  Using cached fastapi-0.115.8-py3-none-any.whl.metadata (27 kB)
  Using cached onnxruntime-1.20.1-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (4.5 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.29.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached opentelemetry_instrumentation_fastapi-0.50b0-py3-none-any.whl.metadata (2.1 kB)
  Using cached kubernetes-32.0.0-py2.py3-none-any.whl.metadata (1.5 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached pydantic_settings-2.7.1-py3-none-any.whl.metadata (3.5 kB)
  Using cached coloredlogs-15.0.1-py2.py3-none-any.whl.metadata (12 kB)
  Using cached opentelemetry_exporter_otlp_proto_common-1.29.0-py

In [ ]:
!pip install -U langchain-community

In [ ]:
!pip install langchain-huggingface

In [ ]:
!pip install sentence-transformers

In [ ]:
!pip install faiss-gpu-cu11

In [ ]:
!pip install pypdf

*Importing* *Packages*

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_huggingface import HuggingFaceEmbeddings

Import THE Document

In [ ]:
# Load document using PyPDFLoader document loader
# LLM dont know our personal information. So I am using my resume here.
# Upload the document under files section. The file name can be changed here.
loader = PyPDFLoader("/content/Resume.pdf")
documents = loader.load()

In [ ]:
len(docs)


17

CHUNKING

In [ ]:
#Splitting the data into chunk
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=30, separator="\n")
docs = text_splitter.split_documents(documents=documents)

print(docs)

In [ ]:
len(chunks)

96

In [ ]:
print(chunks)

[Document(metadata={'source': '/content/drive/MyDrive/Hospital BOT/Data .pdf', 'page': 0, 'page_label': '1'}, page_content='Brihadeeswarar Temple: Full History & Details \nOverview \nBrihadeeswarar Temple, also known as Peruvudaiyar Kovil, is one of the greatest \narchitectural marvels of India. It is a Hindu temple dedicated to Lord Shiva and is located in'), Document(metadata={'source': '/content/drive/MyDrive/Hospital BOT/Data .pdf', 'page': 0, 'page_label': '1'}, page_content='Thanjavur, Tamil Nadu, India. The temple is part of the Great Living Chola Temples, a \nUNESCO World Heritage Site. \nHistorical Background \n● Built By: Rajaraja Chola I (985–1014 CE) \n● Construction Year: Completed in 1010 CE \n● Architectural Style: Dravidian Architecture'), Document(metadata={'source': '/content/drive/MyDrive/Hospital BOT/Data .pdf', 'page': 0, 'page_label': '1'}, page_content='● UNESCO World Heritage Site: Since 1987 \nArchitectural Features \n● The temple is built entirely out of grani

In [ ]:
chunks[50]

Document(metadata={'source': '/content/drive/MyDrive/Hospital BOT/Data .pdf', 'page': 8, 'page_label': '9'}, page_content='● Distance from Thanjavur: 35 km \n● Distance from Chennai: 290 km \n \nAverage Visitors Per Year \nAiravatesvara Temple is a significant pilgrimage site and attracts history and architecture \nenthusiasts. \n● Total Yearly Visitors: 0.8 to 1 million people')

Embeddings Creation


In [ ]:
#loading the embedding model from huggingface
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(
  model_name=embedding_model_name,
  model_kwargs=model_kwargs
)

<ipython-input-27-059dc5d344e3>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=SentenceTransformerEmbeddings(model_name="NeuML/pubmedbert-base-embeddings")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or d

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/6.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/706k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Persist the vectors locally on disk
vectorstore.save_local("faiss_index_")

In [ ]:
# Load from local storage
persisted_vectorstore = FAISS.load_local("faiss_index_", embeddings,allow_dangerous_deserialization=True)

In [ ]:
#creating a retriever on top of database
retriever = persisted_vectorstore.as_retriever()

In [ ]:
retriever= vectorstore.as_retriever(search_kwargs={"k":5})

In [ ]:
retriever.get_relevant_documents(query)

<ipython-input-16-162c3489abc9>:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents(query)


[Document(metadata={'page': 8, 'source': '/content/drive/MyDrive/Hospital BOT/healthyheart.pdf'}, page_content='While each risk factor increases your risk of heart disease, having'),
 Document(metadata={'page': 5, 'source': '/content/drive/MyDrive/Hospital BOT/healthyheart.pdf'}, page_content='■As early as age 45, a man’s risk of heart disease begins to rise significantly. For a woman, risk starts to increase at age 55.\n■Fifty percent of men and 64 percent of women who die suddenlyof heart disease have no previous symptoms of the disease.1Heart Disease: Why Should You Care?Heart Disease:'),
 Document(metadata={'page': 6, 'source': '/content/drive/MyDrive/Hospital BOT/healthyheart.pdf'}, page_content='factor greatly increases the chances of developing heart disease.Moreover, the worse a particular risk factor is, the more likely youare to develop heart disease. For example, if you have high bloodpressure, the higher it is, the greater your chances of developingheart disease, including 

**LLM Loading**

In [ ]:
!pip install langchain_ollama

In [ ]:
!pip install colab-xterm #https://pypi.org/project/colab-xterm/
%load_ext colabxterm

In [ ]:
%xterm

**DEEEP SEEK**

In [ ]:
%xterm

## ollama run deepseek-r1

In [ ]:
from langchain_community.llms import Ollama

In [ ]:
# Initialize an instance of the Ollama model
llm = Ollama(model="deepseek-r1")

In [ ]:
# Invoke the model to generate responses
response = llm.invoke("Who is Arul Benjamin Chandru?")
print(response)

In [2]:
 #Use RetrievalQA chain for orchestration
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

NameError: name 'RetrievalQA' is not defined

In [ ]:
while True:
  query = input("Type your query if you want to exit type Exit: \n")
  if query == "Exit":
    break
  result = qa.run(query)
  print(result)